### **Import Libraries**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim as optim

### **Load Data**

In [ ]:
# Open text file and read in data as 'text'
with open('drive/My Drive/Colab Notebooks/anna.txt', 'r') as f:
  text = f.read()

In [ ]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### **Tokenization**

In [ ]:
#encode the text and map each character to an integer and vice versa

#get each unique text
chars = tuple(set(text))
#maps integers to characters
int2char = dict(enumerate(chars))
#maps characters to unique integers
char2int = {ch: ii for ii, ch in int2char.items()}


#encode the text with an array of integer
encoded = np.array([char2int[ch] for ch in text])



In [ ]:
encoded[:100]

array([23, 74, 22, 17,  6, 31, 32, 21, 75, 10, 10, 10,  7, 22, 17, 17, 81,
       21,  4, 22, 53, 36, 54, 36, 31, 18, 21, 22, 32, 31, 21, 22, 54, 54,
       21, 22, 54, 36, 46, 31, 30, 21, 31, 67, 31, 32, 81, 21, 69, 41, 74,
       22, 17, 17, 81, 21,  4, 22, 53, 36, 54, 81, 21, 36, 18, 21, 69, 41,
       74, 22, 17, 17, 81, 21, 36, 41, 21, 36,  6, 18, 21, 33, 51, 41, 10,
       51, 22, 81, 80, 10, 10, 29, 67, 31, 32, 81,  6, 74, 36, 41])

### **Pre-processing the data**

#### One Hot Encoding

In [ ]:
def one_hot_encode(arr, n_labels):
  #initialize the encoded array
  one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

  #fill the appropriate elements with ones
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

  #finally reshape it to get back the original array
  one_hot = one_hot.reshape((*arr.shape, n_labels))

  return one_hot

In [ ]:
#check that the function works as expected
test_seq = np.array([[3,5,1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


##### Create Batch

In [ ]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


### Test Implementation

In [ ]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [ ]:
print('x\n', x[:10,:10])
print('\ny\n', y[:10,:10])

x
 [[23 74 22 17  6 31 32 21 75 10]
 [18 33 41 21  6 74 22  6 21 22]
 [31 41 66 21 33 32 21 22 21  4]
 [18 21  6 74 31 21  9 74 36 31]
 [21 18 22 51 21 74 31 32 21  6]
 [ 9 69 18 18 36 33 41 21 22 41]
 [21 34 41 41 22 21 74 22 66 21]
 [12 24 54 33 41 18 46 81 80 21]]

y
 [[74 22 17  6 31 32 21 75 10 10]
 [33 41 21  6 74 22  6 21 22  6]
 [41 66 21 33 32 21 22 21  4 33]
 [21  6 74 31 21  9 74 36 31  4]
 [18 22 51 21 74 31 32 21  6 31]
 [69 18 18 36 33 41 21 22 41 66]
 [34 41 41 22 21 74 22 66 21 18]
 [24 54 33 41 18 46 81 80 21 78]]


### Create Model

In [ ]:
class CharRNN(nn.Module):
  def __init__(self, tokens, n_hidden=256, n_layers=2,
                             drop_prob=0.5, lr=0.001):
    super().__init__()
    self.drop_prob = drop_prob
    self.n_layers = n_layers
    self.n_hidden = n_hidden
    self.lr = lr

    # creating character dictionaries
    self.chars = tokens
    self.int2char = dict(enumerate(self.chars))
    self.char2int = {ch: ii for ii,ch in self.int2char.items()}

    #define the LSTM
    self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                        dropout=drop_prob, batch_first=True)
    
    #define a dropout layer
    self.dropout = nn.Dropout(drop_prob)

    #define the final, fully-connected output layer
    self.fc = nn.Linear(n_hidden, len(self.chars))

  def forward(self, x, hidden):
      ''' Forward pass throuh the network.
      These inputs are x, and the hidden/cell state 'hidden'. '''

      # get the outputs and the new hidden state from the lstm
      r_output, hidden = self.lstm(x, hidden)

      # pass through a dropout layer
      out = self.dropout(r_output)

      # Stack up LSTM outputs using view
      out = out.contiguous().view(-1, self.n_hidden)

      #put x through the fully-connected layer
      out = self.fc(out)


      #return the final output and the hidden state
      return out, hidden

  
  def init_hidden(self, batch_size):
    '''Initializes hidden stae'''
    #create two new tensors with sizes n_layers x batch_size x n_hidden,
    #initialized to zero, for hidden state and cell state of LSTM
    weight = next(self.parameters()).data
    
    hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
              weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
    
    return hidden

### Train the Model

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
  ''' Arguments
      ---------
      net: CharRNN network
      data: text data to train the network
      batch_size: Number of mini-sequence per mini-batch
      seq_length: Number of character steps per mini-batch
      lr: learning rate
      clip: gradient clippin
      val_frac: Fraction of data to hold out for validation
      print every: Number of steps fpr printing training and validation loss'''

  opt = optim.Adam(net.parameters(), lr=lr)
  criterion = nn.CrossEntropyLoss()

  #create training and validation data
  val_idx = int(len(data)*(1-val_frac))
  data, val_data = data[:val_idx], data[val_idx:]

  counter = 0
  n_chars = len(net.chars)
  for e in range(epochs):
    #initialize hidden state
    h = net.init_hidden(batch_size)

    for x, y in get_batches(data, batch_size, seq_length):
      counter += 1

      # one-hot encode our data and make them Torch tensors
      x = one_hot_encode(x, n_chars)
      inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

      #creatin new variable for the hidden state, otherwise
      #we'd backprop through the entire trainin history

      h = tuple([each.data for each in h])

      #zero accumulated gradients
      net.zero_grad()

      #get output from model
      output, h = net(inputs, h)

      #calcualte the loss and perform backprop
      loss = criterion(output, targets.view(batch_size*seq_length))
      loss.backward()

      #clip_grad_norm helps prevent the exploding gradient problem in RNN/LSTM
      nn.utils.clip_grad_norm_(net.parameters(), clip)
      opt.step()

      #loss stats
      if counter % print_every == 0:
        #get validation loss
        val_h = net.init_hidden(batch_size)
        val_losses = []
        for x, y in get_batches(val_data, batch_size, seq_length):
          #one-hot encode data and make them torch tensors
          x = one_hot_encode(x, n_chars)
          x, y = torch.from_numpy(x), torch.from_numpy(y)
        
          # Creating new variables for the hidden state, otherwise
          # we'd backprop through the entire training history
          val_h = tuple([each.data for each in val_h])
                    
          inputs, targets = x, y
        
          inputs, targets = inputs, targets

          output, val_h = net(inputs, val_h)
          val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
          val_losses.append(val_loss.item())
                
          net.train() # reset to train mode after iterationg through validation data
                
          print("Epoch: {}/{}...".format(e+1, epochs),
                "Step: {}...".format(counter),
                "Loss: {:.4f}...".format(loss.item()),
                "Val Loss: {:.4f}".format(np.mean(val_losses)))

### **Instantiate Model**

In [ ]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 15 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.3104
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.3013
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2994
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2983
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.3012
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2940
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2902
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2868
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2856
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2865
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2857
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2880
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2861
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2861
Epoch: 1/15... Step: 10... Loss: 3.2822... Val Loss: 3.2859
Epoch: 1/15... Step: 20... Loss: 3.1576... Val Loss: 3.1969
Epoch: 1/15... Step: 20... Loss: 3.1576.

In [ ]:
save_path = 'drive/MyDrive/Colab Notebooks/net.pth'
torch.save(net.state_dict, save_path)

### **Checkpoint**

In [ ]:
# change the name, for saving multiple files
model_name = 'rnn_15_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

### **Make Prediction**

In [ ]:
def predict(net, char, h=None, top_k=None):
  ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
  # tensor inputs
  x = np.array([[net.char2int[char]]])
  x = one_hot_encode(x, len(net.chars))
  inputs = torch.from_numpy(x)
        
      
        
  # detach hidden state from history
  h = tuple([each.data for each in h])
  # get the output of the model
  out, h = net(inputs, h)

  # get the character probabilities
  p = F.softmax(out, dim=1).data
    
        
  # get top characters
  if top_k is None:
    top_ch = np.arange(len(net.chars))
  else:
    p, top_ch = p.topk(top_k)
    top_ch = top_ch.numpy().squeeze()
        
    # select the likely next character with some element of randomness
  p = p.numpy().squeeze()
  char = np.random.choice(top_ch, p=p/p.sum())
        
  # return the encoded value of the predicted char and the hidden state
  return net.int2char[char], h

### **Prime and Generate Text**

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
  net.eval() # eval mode
    
  # First off, run through the prime characters
  chars = [ch for ch in prime]
  h = net.init_hidden(1)
  for ch in prime:
    char, h = predict(net, ch, h, top_k=top_k)

  chars.append(char)
    
    # Now pass in the previous character and get a new one
  for ii in range(size):
    char, h = predict(net, chars[-1], h, top_k=top_k)
    chars.append(char)

  return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, and treating him. "You've
come thought as, the croase, and then you're taken im to see your
proper of my, but I'm general abready."

"Why is, you've been more threakenses, and I've say any more, and have a give
moment to be self."

"When you were not, it's all all that I does are so meant to the ceasing that
their pity and the peasants of countes and meets?"

"Oh, you meant! I could stay a man, we danger as he was so a man of men. It's a
second mother with myself on the peasants. I don't say that," she added,
with his sense in the peasant, he saw what she was taking in the consideration. As they said to her
than and to be a good and attaired to the table. He was nevired the same
thing, his from the marsh had talked to be sering a motern of some
carming on to her. They were stonition, and with a freenor
on the seminite position to be sud all steps on the starter of seeming.

"I won't the men, I went to stop him, I'm
at once to see the capital of my child are trustful about the can

### **Load Checkpoint**

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_15_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
he had sat difficult see in at it.

"I walk a great means to take her. Where are you thinks. I should be so as all
there all was to see this that."

Alexey Alexandrovitch seemed, and heart toos he would be thanding horrily and things
that he was answer and sounds. Seryozha. At the same things of the calmest and at
an arm things and feeling of seesed his brother, and the mother that she
was the baby time, and went out of sincere.

"I have no morning and so telling the significance of myself without
my bing and means, the man, I've been dear in the foots and
simply our feeling of them. Tis seciniment too. I'm taken to
say it."

"Oh! It are so much and happy to the looking, and wanted to see alr marsh.
This went in this care to him," he said to him.

"I don't know her father's easy," he added, said to his brother's
admather, and that staying with the same tone of his
wife, send on the stury, and shinking off his face and treeting
some for the posetter of her assent face tha